# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [8]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chifeng,CN,42.2683,118.9636,31.80,18,0,5.66,clear sky
1,1,Juegang,CN,32.3174,121.1855,55.98,71,0,15.68,clear sky
2,2,Olafsvik,IS,64.8945,-23.7142,31.75,67,100,21.85,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,63.81,84,81,5.41,broken clouds
4,4,Tilichiki,RU,60.4667,166.1000,-12.93,91,91,10.11,overcast clouds
5,5,Cape Town,ZA,-33.9258,18.4232,65.10,87,0,5.75,clear sky
6,6,Hobart,AU,-42.8794,147.3294,77.27,73,40,23.02,scattered clouds
7,7,Kamenka,RU,51.3223,42.7678,1.76,97,95,3.29,overcast clouds
8,8,Coahuayana,MX,18.7333,-103.6833,69.87,90,0,3.51,clear sky
9,9,Saskylakh,RU,71.9167,114.0833,-25.71,98,100,6.38,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [9]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 75
What is the maximum temperature you would like for your vacation? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [10]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Hobart,AU,-42.8794,147.3294,77.27,73,40,23.02,scattered clouds
10,10,Atuona,PF,-9.8000,-139.0333,78.71,75,5,14.29,clear sky
11,11,Hithadhoo,MV,-0.6000,73.0833,81.52,75,62,10.09,broken clouds
13,13,Utete,TZ,-7.9860,38.7580,77.43,91,33,0.18,scattered clouds
22,22,Sibolga,ID,1.7427,98.7792,82.99,63,100,3.62,overcast clouds
27,27,Pangai,TO,-19.8000,-174.3500,87.91,70,75,5.75,broken clouds
28,28,New Norfolk,AU,-42.7826,147.0587,76.91,44,1,1.99,clear sky
30,30,Avarua,CK,-21.2078,-159.7750,77.05,88,75,17.27,light rain
32,32,Jamame,SO,0.0722,42.7506,80.80,70,77,13.11,broken clouds
35,35,Hilo,US,19.7297,-155.0900,79.29,76,100,5.75,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [11]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Hobart,AU,-42.8794,147.3294,77.27,73,40,23.02,scattered clouds
10,10,Atuona,PF,-9.8000,-139.0333,78.71,75,5,14.29,clear sky
11,11,Hithadhoo,MV,-0.6000,73.0833,81.52,75,62,10.09,broken clouds
13,13,Utete,TZ,-7.9860,38.7580,77.43,91,33,0.18,scattered clouds
22,22,Sibolga,ID,1.7427,98.7792,82.99,63,100,3.62,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [28]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, current description, 
# and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
6,Hobart,AU,77.27,scattered clouds,-42.8794,147.3294
10,Atuona,PF,78.71,clear sky,-9.8000,-139.0333
11,Hithadhoo,MV,81.52,broken clouds,-0.6000,73.0833
13,Utete,TZ,77.43,scattered clouds,-7.9860,38.7580
22,Sibolga,ID,82.99,overcast clouds,1.7427,98.7792
27,Pangai,TO,87.91,broken clouds,-19.8000,-174.3500
28,New Norfolk,AU,76.91,clear sky,-42.7826,147.0587
30,Avarua,CK,77.05,light rain,-21.2078,-159.7750
32,Jamame,SO,80.80,broken clouds,0.0722,42.7506
35,Hilo,US,79.29,overcast clouds,19.7297,-155.0900


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [30]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hobart,AU,77.27,scattered clouds,-42.8794,147.3294,
10,Atuona,PF,78.71,clear sky,-9.8000,-139.0333,
11,Hithadhoo,MV,81.52,broken clouds,-0.6000,73.0833,
13,Utete,TZ,77.43,scattered clouds,-7.9860,38.7580,
22,Sibolga,ID,82.99,overcast clouds,1.7427,98.7792,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [31]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [32]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Hobart - nearest hotel: Vibe Hotel Hobart
Atuona - nearest hotel: Pearl Resort
Hithadhoo - nearest hotel: Pebbles Inn
Utete - nearest hotel: No hotel found
Sibolga - nearest hotel: Hotel Prima Indah
Pangai - nearest hotel: Billy's Place
New Norfolk - nearest hotel: Woodbridge on the Derwent
Avarua - nearest hotel: Paradise Inn
Jamame - nearest hotel: No hotel found
Hilo - nearest hotel: Dolphin Bay Hotel
Daru - nearest hotel: Kuki
Nabire - nearest hotel: Hotel Anggrek
Mataura - nearest hotel: No hotel found
Busselton - nearest hotel: No hotel found
Oistins - nearest hotel: The Abidah By Accra
Faanui - nearest hotel: Oa Oa Lodge
Flinders - nearest hotel: No hotel found
Kapaa - nearest hotel: Pono Kai Resort
Anloga - nearest hotel: Hotel de White House
Bengkulu - nearest hotel: Dena Hotel
Mahebourg - nearest hotel: Grand Bel Air
Aitape - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Catamarca - nearest hotel: Hotel de Turismo
Acapulco -

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hobart,AU,77.27,scattered clouds,-42.8794,147.3294,Vibe Hotel Hobart
10,Atuona,PF,78.71,clear sky,-9.8000,-139.0333,Pearl Resort
11,Hithadhoo,MV,81.52,broken clouds,-0.6000,73.0833,Pebbles Inn
13,Utete,TZ,77.43,scattered clouds,-7.9860,38.7580,No hotel found
22,Sibolga,ID,82.99,overcast clouds,1.7427,98.7792,Hotel Prima Indah
...,...,...,...,...,...,...,...
616,Isla Mujeres,MX,75.16,clear sky,21.2311,-86.7310,Villa La Bella
638,Kulhudhuffushi,MV,80.91,clear sky,6.6221,73.0700,No hotel found
658,Kavieng,PG,84.20,broken clouds,-2.5744,150.7967,Kavieng Hotel
661,Lantawan,PH,78.49,light rain,10.6167,123.9833,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [34]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hobart,AU,77.27,scattered clouds,-42.8794,147.3294,Vibe Hotel Hobart
10,Atuona,PF,78.71,clear sky,-9.8000,-139.0333,Pearl Resort
11,Hithadhoo,MV,81.52,broken clouds,-0.6000,73.0833,Pebbles Inn
13,Utete,TZ,77.43,scattered clouds,-7.9860,38.7580,No hotel found
22,Sibolga,ID,82.99,overcast clouds,1.7427,98.7792,Hotel Prima Indah
27,Pangai,TO,87.91,broken clouds,-19.8000,-174.3500,Billy's Place
28,New Norfolk,AU,76.91,clear sky,-42.7826,147.0587,Woodbridge on the Derwent
30,Avarua,CK,77.05,light rain,-21.2078,-159.7750,Paradise Inn
32,Jamame,SO,80.80,broken clouds,0.0722,42.7506,No hotel found
35,Hilo,US,79.29,overcast clouds,19.7297,-155.0900,Dolphin Bay Hotel


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [36]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [38]:
# Configure the map plot
clean_hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
clean_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)